<a href="https://colab.research.google.com/github/dwhite25/MSG-CcGAN/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/GWPAC/MSG-cGAN/Ours/classes')
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 15.8 MB/s 


In [3]:
import tensorflow as tf 
import dataset as ds
import msggan2 as gan
from pathlib import Path
import zipfile
import csv

In [4]:
# get all 10Gb of sims into the runtime before we start having to do everything else
# this should make the program run much faster than having to load each file just-in-time

zippath = 'drive/MyDrive/GWPAC/MSG-cGAN/Ours/ins/time_series/has_postmergers/sims2.zip'

print('unzipping files in runtime environment. this will take ~10 minutes.')

sims = zipfile.ZipFile(zippath, 'r')
sims.extractall()
sims.close()

print('...done.')

unzipping files in runtime environment. this will take ~10 minutes.
...done.


In [5]:
# define key variables/arguments

loadweights = False
usebias     = True
nrns        = 64
glr         = 1e-2
dlr         = 1e-2
r1g         = 1e0
epsln       = 1e-5
res         = 8192
bsize       = 32
ksize       = 9
b1          = 0.3
b2          = 0.8
metadata    = 'drive/MyDrive/GWPAC/MSG-cGAN/Ours/ins/time_series/has_postmergers/METADATA.csv'

In [ ]:
pathway = 'content/sims'
outpath = f'drive/MyDrive/GWPAC/MSG-cGAN/Ours/outs/ccganb_0.005sd_{nrns}n_{glr}_{dlr}_{r1g}r_{epsln}e_{ksize}k_5'
weights = f'drive/MyDrive/GWPAC/MSG-cGAN/Ours/outs/ccganb_0.005sd_{nrns}n_{glr}_{dlr}_{r1g}r_{epsln}e_{ksize}k_4/weights'
epoch = str(63000)

# save some of the data about the run to the attrs file, in case we want to look this info up later
Path(outpath).mkdir(exist_ok=True)
with open(outpath + '/attrs.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['loadweights', loadweights])
    writer.writerow(['usebias', usebias])
    writer.writerow(['neurons', nrns])  
    writer.writerow(['g learning rate', glr])
    writer.writerow(['d learning rate', dlr])
    writer.writerow(['r1gamma', r1g])
    writer.writerow(['epsilon', epsln])
    writer.writerow(['resolution', res])
    writer.writerow(['batch size', bsize])
    writer.writerow(['beta1', b1])
    writer.writerow(['beta2', b2])

# load the dataset
dataset = ds.Dataset(batch_size=bsize, pathway=pathway, mdpath=metadata, outpath=outpath, nlabels=4, endres=res)

# call the GAN class
msgg = gan.MSG_CcGAN_ts(neurs=nrns, endres=res, g_lr=glr, d_lr=dlr, r1_gamma=r1g, epsilon=epsln, 
                        outpath=outpath, epochs=500000, nchannels=2, ksize=ksize, usebias=usebias,
                        beta1=b1, beta2=b2)

# continue from previous save point?
if loadweights == True:
    msgg.D.load_weights(weights + '/discriminator' + epoch + '.h5')
    msgg.G.load_weights(weights + '/generator' + epoch + '.h5')

# start the training loop...
msgg.train(dataset)

# ... or make sample waveforms 
# lbls = [2858.0, 2858.0, 1.0, 1.0]       # ~TS00001.csv
# lbls = [560.0, 560.0, 1.2, 1.2]         # ~TS00229.csv
# msgg.makewaves(dataset, epoch, lbls)

Total objects found:  50141
Model: "Generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 labels (InputLayer)            [(None, 4)]          0           []                               
                                                                                                  
 reshape_2 (Reshape)            (None, 1, 4)         0           ['labels[0][0]']                 
                                                                                                  
 dense_eq_2 (DenseEQ)           (None, 1, 64)        320         ['reshape_2[0][0]']              
                                                                                                  
 tf.identity_1 (TFOpLambda)     (None, 1, 64)        0           ['dense_eq_2[0][0]']             
                                                              